In [1]:
import json
import pandas as pd
saoke = []
for line in open('../data/SAOKE_DATA.json', 'r'):
    saoke.append(json.loads(line))

def replaceMissinfo(aaa):
    placeholder = ['Z','Y','X']
    for i in range(len(aaa)):
        if aaa[i] == '_':
            aaa = aaa[:i] + placeholder.pop() + aaa[i+1:]
    return aaa
def replaceMisspred(aaa):
    if aaa == '_':
        return 'P'
    else:
        return aaa
    
import jieba
sentences = []
labels = []
original = []
original_label = []
for sentence in saoke:
    if sentence['logic'] == []:
        continue
    seg_list = jieba.cut(sentence['natural'], cut_all=False)
    sentences.append(list(seg_list))
    original.append(sentence)
    sent = ''
    for o in sentence['logic']:
        sent += str(tuple([replaceMissinfo(o['subject'])]+ [replaceMisspred(o['predicate'])] + [replaceMissinfo(o['object'][0])]))
        sent += '$'
        sent = sent.replace("'","").replace(",","@").replace(" ","").replace("(","").replace(")","")
    seg_list = jieba.cut(sent[:-1], cut_all=False)
    original_label.append(sent[:-1])
    labels.append(list(seg_list))

Building prefix dict from the default dictionary ...
Loading model from cache /state/partition1/job-346663/jieba.cache
Loading model cost 0.768 seconds.
Prefix dict has been built succesfully.


In [31]:
vocab_prefix = ['PAD', 'UNK_token', 'EOS', 'SOS']
UNK_token = 0

In [26]:
import numpy as np
from collections import Counter
from itertools import dropwhile



class Lang:
    def __init__(self, name, emb_pretrained_add, max_vocab_size = None):
        self.name = name
        self.word2index = None #{"$PAD$": PAD_token, "$SOS$": SOS_token, "$EOS$": EOS_token, "$UNK$": UNK_token}
        #self.word2count = None #{"$PAD$": 0, "$SOS$" : 0, "$EOS$": 0, "$UNK$": 0}
        self.index2word = None #{PAD_token: "$PAD$", SOS_token: "$SOS$", EOS_token: "$EOS$", UNK_token: "$UNK$"}
        self.max_vocab_size = max_vocab_size  # Count SOS and EOS
        self.vocab_size = None
        self.emb_pretrained_add = emb_pretrained_add
        self.embedding_matrix = None

    def build_vocab(self, train_data, label_data):
        all_tokens = []
        for sent in train_data:
            all_tokens.extend(sent)
        for sent in label_data:
            all_tokens.extend(sent)    
        token_counter = Counter(all_tokens)
        print('The number of unique tokens totally in dataset: ', len(token_counter))
        for key, count in dropwhile(lambda key_count: key_count[1] > 1, token_counter.most_common()):
            del token_counter[key]
        if self.max_vocab_size:
            vocab, count = zip(*token_counter.most_common(self.max_vocab_size))
        else:
            vocab, count = zip(*token_counter.most_common())
        self.index2word = vocab_prefix + list(vocab)
        word2index = dict(zip(vocab, range(len(vocab_prefix),len(vocab_prefix)+len(vocab)))) 
        for idx, token in enumerate(vocab_prefix):
            word2index[token] = idx
        self.word2index = word2index
        return None 

    def build_emb_weight(self):
        words_emb_dict = load_emb_vectors(self.emb_pretrained_add)
        vocab_size = len(self.index2word)
        self.vocab_size = vocab_size
        emb_weight = np.zeros([vocab_size, 300])
        for i in range(len(vocab_prefix), vocab_size):
            emb = words_emb_dict.get(self.index2word[i], None)
            if emb is not None:
                try:
                    emb_weight[i] = emb
                except:
                    pass
                    #print(len(emb), self.index2word[i], emb)
        self.embedding_matrix = emb_weight
        return None

def load_emb_vectors(fasttest_home):
    max_num_load = 500000
    words_dict = {}
    with open(fasttest_home) as f:
        for num_row, line in enumerate(f):
            if num_row >= max_num_load:
                break
            s = line.split()
            words_dict[s[0]] = np.asarray(s[1:])
    return words_dict

In [66]:
UNK_token = OOV = 1
EOS_token = 2

def text2index(data, word2index):
    indexdata = []
    for line in data:
        indexdata.append([word2index[c] if c in word2index.keys() else UNK_token for c in line])
        #indexdata[-1].append(EOS_token)
    print('finish')
    return indexdata

def construct_Lang(name, train_data, train_label, max_vocab_size = None, emb_pretrained_add = None):
    lang = Lang(name, max_vocab_size, emb_pretrained_add)
    lang.build_vocab(train_data, train_label)
    if emb_pretrained_add:
        lang.build_emb_weight()
    return lang

def text2symbolindex(data, word2index):
    indexdata = []
    for line in data:
        indexdata.append([word2index[c] if c in word2index.keys() else OOV for c in line])
        #indexdata[-1].append(EOS_token)
    print('finish')
    return indexdata

In [28]:
trainLang = construct_Lang('train', sentences, labels)

The number of unique tokens totally in dataset:  86302


In [32]:
train_input_index = text2index(sentences, trainLang.word2index) 

finish


In [46]:
label_input_index = text2index(labels, trainLang.word2index) 

finish


In [62]:
symbol_dict = ['PAD','OOV','EOS','ISA','DESC','IN','BIRTH',"DEATH", "=","$", "[","]","|","X","Y","Z","P","@"]

In [50]:
# symbol_index = []
# for o in symbol_dict:
#     symbol_index.append(trainLang.word2index[o])

In [63]:
word2symbolindex =  {}
for idx, token in enumerate(symbol_dict):
        word2symbolindex[token] = idx

In [67]:
label_input_symbolindex = text2symbolindex(labels, word2symbolindex)

finish


In [79]:
# Check valid
# for idx,i in enumerate(label_input_symbolindex[0]):
#     if symbol_dict[i] == 'OOV':
#         continue
#     elif symbol_dict[i] == labels[0][idx]:
#         continue
#     else:
#         print(symbol_dict[i])

In [108]:
def copy_indicator(sentences,labels):
    indicator = []
    for i in range(len(sentences)):
        tgt = labels[i]
        src = sentences[i]
        matrix = np.zeros((len(tgt)+1,len(src)+1))
        for m in range(len(tgt)):
            for n in range(len(src)):
                if tgt[m] == src[n]:
                    matrix[m,n] = 1
        matrix[len(tgt),len(src)] = 1
        indicator.append(matrix)
    return indicator

In [109]:
indicator = copy_indicator(sentences,labels)

In [129]:
len(train_input_index),len(label_input_index),len(label_input_symbolindex),len(indicator),len(sentences),len(labels)

(40806, 40806, 40806, 40806, 40806, 40806)

In [173]:
EOS_token = 2

In [189]:
import numpy as np
import time
import os.path
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence


class VocabDataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """
    def __init__(self, src_index, tgt_index, tgt_symbolindex, tgt_indicator, src_original, tgt_original, src_clip = None, tgt_clip = None):
        """
        @param data_list: list of character
        @param target_list: list of targets

        """
        self.src_clip = src_clip
        self.tgt_clip = tgt_clip
        self.src_list, self.tgt_list = src_index, tgt_index
        self.src_original, self.tgt_original = src_original, tgt_original
        self.tgt_symbolindex, self.tgt_indicator  = tgt_symbolindex, tgt_indicator
        
        assert (len(self.src_list) == len(self.tgt_list)== len(self.src_original)== len(self.tgt_original)== len(self.tgt_symbolindex)== len(self.tgt_indicator))
        #self.word2index = word2index

    def __len__(self):
        return len(self.src_list)

    def __getitem__(self, key):
        """
        Triggered when you call dataset[i]
        """
        
        src = self.src_list[key]
        tgt = self.tgt_list[key]
        src_org = self.src_original[key]
        tgt_org = self.tgt_original[key]
        tgt_sym = self.tgt_symbolindex[key]
        tgt_ind = self.tgt_indicator[key]
        
        if self.src_clip is not None:
            src = src[:self.src_clip]
            src_org = src_org[:self.src_clip]
            tgt_ind = tgt_ind[:,:self.src_clip]
        src_length = len(src)

        if self.tgt_clip is not None:
            tgt = tgt[:self.tgt_clip]
            tgt_org = tgt_org[:self.tgt_clip]
            tgt_sym = tgt_sym[:self.tgt_clip]
            tgt_ind = tgt_ind[:self.tgt_clip,:]
        tgt_length = len(tgt)
        
        return src,src_length,tgt,tgt_length, tgt_sym, tgt_ind, src_org, tgt_org, 
        
        #return src_org, src_tensor, src_true_len, tgt_org, tgt_tensor, tgt_true_len, tgt_label_vocab, tgt_label_copy 


def vocab_collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all
    data have the same length
    """
    src_list = []
    tgt_list = []
    src_length_list = []
    tgt_length_list = []
    tgt_symbol_list = []
    tgt_indicator_list = []
    src_org_list = []
    tgt_org_list = []
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for datum in batch:
        src_length_list.append(datum[1]+1)
        tgt_length_list.append(datum[3]+1)
    
    batch_max_src_length = np.max(src_length_list)
    batch_max_tgt_length = np.max(tgt_length_list)
    # padding
    for datum in batch:
        padded_vec = np.pad(np.array(datum[0]+[EOS_token]),
                                pad_width=((0, batch_max_src_length-datum[1]-1)),
                                mode="constant", constant_values=0)
        src_list.append(padded_vec)
        
        padded_vec = np.pad(np.array(datum[2]+[EOS_token]),
                                pad_width=((0, batch_max_tgt_length-datum[3]-1)),
                                mode="constant", constant_values=0)
        tgt_list.append(padded_vec)
        
        padded_vec = np.pad(np.array(datum[4]+[EOS_token]),
                                pad_width=((0, batch_max_tgt_length-datum[3]-1)),
                                mode="constant", constant_values=0)
        tgt_symbol_list.append(padded_vec)
        
        padded_vec = np.pad(np.array(datum[5]),
                                pad_width=((0, batch_max_tgt_length-datum[3]-1),((0, batch_max_src_length-datum[1]-1))),
                                mode="constant", constant_values=0)
        tgt_indicator_list.append(padded_vec)
        
        src_org_list.append(datum[6])
        tgt_org_list.append(datum[7])
    
    # re-order
#     ind_dec_order = np.argsort(src_length_list)[::-1]
#     data_list = np.array(data_list)[ind_dec_order]
#     train_length_list = np.array(train_length_list)[ind_dec_order]
#     label_list = np.array(label_list)[ind_dec_order]
#     label_length_list = np.array(label_length_list)[ind_dec_order]

    src_list = np.array(src_list)
    src_length_list = np.array(src_length_list)
    tgt_list = np.array(tgt_list)
    tgt_length_list = np.array(tgt_length_list)
    tgt_symbol_list = np.array(tgt_symbol_list)
    tgt_indicator_list = np.array(tgt_indicator_list)
    
    #print(type(np.array(data_list)),type(np.array(label_list)))
    
    return [torch.from_numpy(src_list).to(device), 
            torch.LongTensor(src_length_list).to(device), 
            torch.from_numpy(tgt_list).to(device), 
            torch.LongTensor(tgt_length_list).to(device),
            torch.from_numpy(tgt_symbol_list).to(device),
            torch.from_numpy(tgt_indicator_list).to(device),
            src_org_list,
            tgt_org_list,           
           ]




In [190]:
train_dataset = VocabDataset(train_input_index, label_input_index, label_input_symbolindex, indicator, sentences, labels)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=1,
                                               collate_fn=vocab_collate_func,
                                               shuffle=False)

In [191]:
for src_tensor, src_true_len, tgt_tensor, tgt_true_len, tgt_label_vocab, tgt_label_copy, src_org, tgt_org in train_loader:
    print(src_tensor[:10])
    print(src_true_len)
    print(tgt_tensor[:10])
    print(tgt_true_len)
    print(tgt_label_vocab[:10])
    print(tgt_label_copy[:10,:10])
    print(src_org[:10])
    print(tgt_org[:10])
    break

[[   66  5126    59 30342    16 14787  4429     3 12152  5636     9  5636
     16  4430 21069 21070    35     5    43  5015    59 23081    16 26988
      5  5637    15  3122  3324 45328    50  1020 23082   232     3    35
      9  3273  4429     5    15  3325  5637   891    46    50  1020  1969
  12933     3  1970    35     5  4897 45329    50 14787    16 13724 18827
      3 12152  5636     9 23081    16 21071  9141 45330     5  3324    43
  45331  9531     9 18828    16     0 45332     9 45333 30343    16 13725
     35     3  9142     8     2]]
tensor([[   66,  5126,    59, 30342,    16, 14787,  4429,     3, 12152,  5636,
             9,  5636,    16,  4430, 21069, 21070,    35,     5,    43,  5015,
            59, 23081,    16, 26988,     5,  5637,    15,  3122,  3324, 45328,
            50,  1020, 23082,   232,     3,    35,     9,  3273,  4429,     5,
            15,  3325,  5637,   891,    46,    50,  1020,  1969, 12933,     3,
          1970,    35,     5,  4897, 45329,    50, 14